# Model Creation

In [1]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25890 sha256=9e5fa0d09c4177706ab4b86eb4671842bc3873928149c55bebbbdb2fd1381324
  Stored in directory: c:\users\prade\appdata\local\pip\cache\wheels\31\d0\2c\87ce38f6052879e5b7b18f0f8b4a10ad2a9d210e908d449f16
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'imutils' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'imutils'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [1]:
pip install opencv-python

   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
    --------------------------------------- 0.8/39.0 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.8/39.0 MB 4.4 MB/s eta 0:00:09
   -- ------------------------------------- 2.6/39.0 MB 4.2 MB/s eta 0:00:09
   --- ------------------------------------ 3.4/39.0 MB 4.1 MB/s eta 0:00:09
   ---- ----------------------------------- 4.5/39.0 MB 4.1 MB/s eta 0:00:09
   ----- ---------------------------------- 5.0/39.0 MB 4.0 MB/s eta 0:00:09
   ------ --------------------------------- 6.0/39.0 MB 4.0 MB/s eta 0:00:09
   ------ --------------------------------- 6.8/39.0 MB 4.0 MB/s eta 0:00:09
   ------- -------------------------------- 7.6/39.0 MB 4.0 MB/s eta 0:00:08
   -------- ------------------------------- 8.4/39.0 MB 4.0 MB/s eta 0:00:08
   --------- ------------------------------ 9.2/39.0 MB 4.0 MB/s eta 0:00:08
   ---------- ----------------------------- 10.0/39.0 MB 4.0 MB/s eta 0:00:08
   --

  You can safely remove it manually.
  You can safely remove it manually.


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

#### **MobileNet library**
for object detection

**Flatten**

In [2]:
INIT_LR = 1e-4 #Initial Learning rate
EPOCHS=20
BS=32 # Batch size

In [3]:
directory = r'C:\Users\prade\OneDrive\Desktop\data_science\Machine Learning\Projects\PrepInsta Video Based Facemask Detection\dataset'
categories = ["with_mask","without_mask"]

In [4]:
# Grab the list of images in our dataset directoryy then initialize

print("[INFO] loading images........")

data=[]
labels=[] # with mask and without mask

for category in categories:
    path = os.path.join(directory, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)

[INFO] loading images........


E:\anaconda\envs\ML\Lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
# Performing One-Hot encoding to labels

lb = LabelBinarizer()

labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)
(trainX,testX,trainY,testY) = train_test_split(data, labels,test_size=0.2, stratify = labels, random_state=42)

In [6]:
# Construct the training image generator for data augmentation

aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [7]:
# Load the MobileNetV2 network, ensuring the head FC layer sets are left off

baseModel = MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))

C:\Users\prade\AppData\Local\Temp\ipykernel_23132\4191542458.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))


In [8]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128,activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2,activation='softmax')(headModel)

In [9]:
# place the head FC model on top of base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

#loop over all layers in the base model and freeze them so they will not be updated during the first training process
for layer in baseModel.layers:
    layer.trainable=False

In [12]:
# Compile our Model
print("[INFO] compiling model.....")

opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])

# Train the head of the network
print("[INFO] Training model........")

H = model.fit(aug.flow(trainX, trainY, batch_size=BS), steps_per_epoch=len(trainX)//BS, validation_data = (testX,testY), validation_steps=len(testX)//BS, epochs=EPOCHS)

[INFO] compiling model.....
[INFO] Training model........


E:\anaconda\envs\ML\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.8563 - loss: 0.3906 - val_accuracy: 0.9791 - val_loss: 0.1458
Epoch 2/20
 1/95 ━━━━━━━━━━━━━━━━━━━━ 1:11 761ms/step - accuracy: 1.0000 - loss: 0.0922

E:\anaconda\envs\ML\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


95/95 ━━━━━━━━━━━━━━━━━━━━ 19s 195ms/step - accuracy: 1.0000 - loss: 0.0922 - val_accuracy: 0.9791 - val_loss: 0.1445
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.9684 - loss: 0.1409 - val_accuracy: 0.9896 - val_loss: 0.0767
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 17s 172ms/step - accuracy: 1.0000 - loss: 0.0982 - val_accuracy: 0.9896 - val_loss: 0.0757
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 88s 932ms/step - accuracy: 0.9763 - loss: 0.0976 - val_accuracy: 0.9935 - val_loss: 0.0516
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 17s 171ms/step - accuracy: 1.0000 - loss: 0.0741 - val_accuracy: 0.9935 - val_loss: 0.0514
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 88s 926ms/step - accuracy: 0.9733 - loss: 0.0798 - val_accuracy: 0.9935 - val_loss: 0.0437
Epoch 8/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 17s 172ms/step - accuracy: 1.0000 - loss: 0.0222 - val_accuracy: 0.9935 - val_loss: 0.0433
Epoch 9/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 88s 927ms/step - accuracy: 0.9832 - loss: 0.0643 - val_accuracy: 0.9935 -

In [16]:
# Make Predictions on the training set
print("[INFO] evaluating network.......")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in testing set we need to find the index of label with corresponding largest prediced probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs, target_names = lb.classes_))

# serialize the model to disk
print("[INFO] Saving mask detector model now.............")
model.save("Mask_detector.keras")

[INFO] evaluating network.......
24/24 ━━━━━━━━━━━━━━━━━━━━ 19s 775ms/step
              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767

[INFO] Saving mask detector model now.............
